In [24]:
def merge_responses(stories_path_list, domain_path, write_domain_path):
    all_responses = []
    for story_path in stories_path_list:
        story_file = open(story_path, 'r')
        story =  story_file.readlines()
        try:
            idx = story.index('responses:\n')
            responses = story[idx+1:]
            for i in responses:
                if i.strip().startswith('#'):
                    responses.remove(i)
            all_responses.append(responses)
            story_file.close()
        except ValueError:
            pass
    all_responses.insert(0, 'responses:\n')

    domain_file = open(domain_path, 'r')
    domain = domain_file.readlines()
    
    try:
        responses_idx, actions_idx = domain.index('responses:\n'), domain.index('actions:\n')
        print(responses_idx, actions_idx)
        domain_pre = domain[:responses_idx]
        domain_post = domain[actions_idx:]
        domain_post.insert(0, '\n\n')

        domain = domain_pre + all_responses + domain_post
    except ValueError:
        pass
    
    save_file = open(write_domain_path, 'w+')
    for line in domain:
        save_file.writelines(line)
    save_file.close()


if __name__ == '__main__':
    stories_path_list = ['/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_start_talk.yml',
                            '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_ask_reason.yml']
    domain_path = '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/domain.yml'
    write_domain_path = '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/domain-test.yml'

    all_responses, domain = merge_responses(stories_path_list, domain_path, write_domain_path)


173 175


In [16]:
domain

["version: '2.0'\n",
 'config:\n',
 '  store_entities_as_slots: true\n',
 'session_config:\n',
 '  session_expiration_time: 60\n',
 '  carry_over_slots_to_new_session: true\n',
 '\n',
 'intents:\n',
 '- greet:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- goodbye:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- bot_challenge:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- affirm:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- deny:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- record:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- start:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- stop:\n',
 '    use_entities: true\n',
 '    ignore_entities:\n',
 '      - sentiment\n',
 '- profile_form:\n',
 '    use

---

In [1]:
import yaml

def load_yaml(load_path):
    responses_file = open(load_path, 'r')
    responses = responses_file.read()
    responses = yaml.safe_load(responses)

    return responses

response_path = '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories.yml'
responses = load_yaml(response_path)

In [15]:
def merge_stories(stories_path_list, write_path):
    all_stories = []
    for story_path in stories_path_list:
        story_file = open(story_path, 'r')
        story =  story_file.readlines()
        for line in story[1:]:
            if line != 'responses:\n':
                all_stories.append(line)
            else:
                break
        story_file.close()
    
    all_stories.insert(0, 'version: "2.0"\n\nstories:\n')
    save_file = open(write_path, 'w+')
    for line in all_stories:
        save_file.writelines(line)
    save_file.close()


stories_path_list = ['/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_mood.yml',
                    '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_weather.yml',
                    '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_step_count.yml',
                    '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_start_talk.yml']

write_path = '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories.yml'

merge_stories(stories_path_list, write_path)

---

In [2]:
# def merge_nlu(nlu_path_list):
#     all_nlu = []
#     for nlu_path in nlu_path_list:
#         nlu_file = open(nlu_path, 'r')
#         nlu = nlu_file.readlines()
#         all_nlu.append(nlu[1:])
#         nlu_file.close()
#     return all_nlu

import yaml
def load_yaml(load_path):
    responses_file = open(load_path, 'r')
    responses = responses_file.read()
    responses = yaml.safe_load(responses)
    return responses


nlu_path_list = ['/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/nlu/nlu_en.yml',
                    '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/nlu/nlu_de.yml']

In [68]:
def merge_nlu(nlu_path_list, write_path):
    nlu = load_yaml(nlu_path_list[0]) 
    df = pd.DataFrame(nlu['nlu'])
    for nlu_path in nlu_path_list[1:]:
        nlu = load_yaml(nlu_path) 
        df_raw = pd.DataFrame(nlu['nlu'])
        df = pd.merge(df, df_raw, on=['intent'])
        df['examples'] = df[['examples_x', 'examples_y']].apply(lambda x: x['examples_x']+x['examples_y'], axis=1)
        df = df.drop(['examples_x', 'examples_y'], axis=1)

    all_nlu = []
    all_nlu.insert(0, 'version: "2.0"\nnlu:\n')
    for i in range(len(df)):
        all_nlu.append(f'- intent: {df.loc[i, "intent"]}\n')
        all_nlu.append(f'  examples: |\n{df.loc[i, "examples"].replace("- ", "    - ")}')

    save_file = open(write_path, 'w+')
    for line in all_nlu:
        save_file.writelines(line)
    save_file.close()

In [69]:
nlu_path_list = ['/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/nlu/nlu_en.yml',
                '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/nlu/nlu_de.yml',
                '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/nlu/nlu_nl.yml']

write_path = 'nlu_test.yml'
merge_nlu(nlu_path_list, write_path)

---

In [7]:
all_responses = []
stories_path_list = ['/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_start_talk.yml',
                        '/Users/xinsun/SunDatabase/OneDrive - UvA/SUN Space/Dev_UvA/timely_bot/data/stories/stories_ask_reason.yml']
for story_path in stories_path_list:
    story_file = open(story_path, 'r')
    story =  story_file.readlines()
    try:
        idx = story.index('responses:\n')
        responses = story[idx+1:]
        all_responses.append(responses)
        story_file.close()
    except ValueError:
        pass

print(f'i is {all_responses}')

# all_stories.insert(0, 'version: "2.0"\n\nstories:\n')
# save_file = open(write_path, 'w+')
# for line in all_stories:
#     save_file.writelines(line)
# save_file.close()




i is [['  utter_self_intro:\n', '  - text: I am your health assistant, hope can help you build healthier lifestyle against the cardiac issues. \n', '  utter_instruction:\n', '  - text: Currently, we can talk about your step count, mood, weather, exercise plan or obesity. Just simply say "2000 steps done!" or "I feel not good" or "sunny day" to me!'], ['  utter_pa_ask_if_like:\n', '  - text: Do you like exercise?\n', '  utter_pa_ask_pro:\n', '  - text: What is good about exercise you think?\n', '  utter_pa_ask_pro_more:\n', '  - text: What else do you enjoy about doing exercise?\n', '  utter_pa_reason_ask_confirm:\n', '  - text: Did I summarize everything back correctly?\n', '  utter_pa_ask_con:\n', '  - text: What is bad about exercise you think?\n', '  utter_pa_next_step:\n', "  - text: Great! Let's go to the next step!\n", '  utter_pa_ask_pro_experience:\n', '  - text: Please describe a time when you experienced stress and ended up doing exercise.\n', '  utter_pa_ask_con_experience:\